# nba-rating

Expected score of Team A:
$$E_A=\frac{1}{1+10^{(R_B-R_A)/400}}$$

Expected score of Team B:
$$E_B=\frac{1}{1+10^{(R_A-R_B)/400}}$$

These are also represented by: 
$$E_A=\frac{Q_A}{Q_A+Q_B}$$
$$E_B=\frac{Q_B}{Q_A+Q_B}$$


$Q_A=10^{R_A/400}$, $Q_B=10^{R_B/400}$

In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
from datetime import datetime
import requests
import bs4
from NBARater import NBARater

In [8]:
# Script for scraping data
!python NBAScraper.py --beginning 2010 --end 2024

Scraping data for the 2010 to 2024 seasons...
Fetching 2010-2011 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2011-2012 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2012-2013 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2013-2014 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2014-2015 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2015-2016 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2016-2017 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2017-2018 season...
Loading: 100%|█████████████████████████████████| 35/35 [00:35<00:00,  1.00s/sec]
Fetching 2018-2019 season...
Loading: 100%|███████████████████████████████

In [10]:
df = pd.read_csv(Path('data')/'2010-2024.csv')
df

,Date,Start Time (ET),Visitor,Visitor Points,Home,Home Points,Box Score,Overtime,Attendance,Arena,Notes
0,2010-10-26,7:30p,Miami Heat,80,Boston Celtics,88,https://www.basketball-reference.com//boxscore...,NaN,"18,624",TD Garden,NaN
1,2010-10-26,10:00p,Phoenix Suns,92,Portland Trail Blazers,106,https://www.basketball-reference.com//boxscore...,NaN,"20,603",Rose Garden Arena,NaN
2,2010-10-26,10:30p,Houston Rockets,110,Los Angeles Lakers,112,https://www.basketball-reference.com//boxscore...,NaN,"18,997",STAPLES Center,NaN
3,2010-10-27,7:00p,Boston Celtics,87,Cleveland Cavaliers,95,https://www.basketball-reference.com//boxscore...,NaN,"20,562",Quicken Loans Arena,NaN
4,2010-10-27,7:00p,New York Knicks,98,Toronto Raptors,93,https://www.basketball-reference.com//boxscore...,NaN,"18,722",Air Canada Centre,NaN
...,...,...,...,...,...,...,...,...,...,...,...
15823,2024-02-29,7:30p,Golden State Warriors,110,New York Knicks,99,https://www.basketball-reference.com//boxscore...,NaN,"19,812",Madison Square Garden (IV),NaN
15824,2024-02-29,8:30p,Oklahoma City Thunder,118,San Antonio Spurs,132,https://www.basketball-reference.com//boxscore...,NaN,"18,392",Frost Bank Center,NaN
15825,2024-02-29,9:00p,Houston Rockets,105,Phoenix Suns,110,https://www.basketball-reference.com//boxscore...,NaN,"17,071",Footprint Center,NaN
15826,2024-02-29,10:00p,Miami Heat,97,Denver Nuggets,103,https://www.basketball-reference.com//boxscore...,NaN,"19,634",Ball Arena,NaN


In [11]:
df['Notes'].value_counts()

Playoffs                  646
In-Season Tournament       67
Play-In Game               18
at Mexico City, Mexico     11
at London, England          9
at Paris, France            2
Name: Notes, dtype: int64

In [12]:
df['Date'] = pd.to_datetime(df['Date'])
df['Win'] = np.where(df['Visitor Points'] > df['Home Points'], 1, np.where(df['Visitor Points'] < df['Home Points'], 0, 0.5))
df

,Date,Start Time (ET),Visitor,Visitor Points,Home,Home Points,Box Score,Overtime,Attendance,Arena,Notes,Win
0,2010-10-26,7:30p,Miami Heat,80,Boston Celtics,88,https://www.basketball-reference.com//boxscore...,NaN,"18,624",TD Garden,NaN,0.0
1,2010-10-26,10:00p,Phoenix Suns,92,Portland Trail Blazers,106,https://www.basketball-reference.com//boxscore...,NaN,"20,603",Rose Garden Arena,NaN,0.0
2,2010-10-26,10:30p,Houston Rockets,110,Los Angeles Lakers,112,https://www.basketball-reference.com//boxscore...,NaN,"18,997",STAPLES Center,NaN,0.0
3,2010-10-27,7:00p,Boston Celtics,87,Cleveland Cavaliers,95,https://www.basketball-reference.com//boxscore...,NaN,"20,562",Quicken Loans Arena,NaN,0.0
4,2010-10-27,7:00p,New York Knicks,98,Toronto Raptors,93,https://www.basketball-reference.com//boxscore...,NaN,"18,722",Air Canada Centre,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
15823,2024-02-29,7:30p,Golden State Warriors,110,New York Knicks,99,https://www.basketball-reference.com//boxscore...,NaN,"19,812",Madison Square Garden (IV),NaN,1.0
15824,2024-02-29,8:30p,Oklahoma City Thunder,118,San Antonio Spurs,132,https://www.basketball-reference.com//boxscore...,NaN,"18,392",Frost Bank Center,NaN,0.0
15825,2024-02-29,9:00p,Houston Rockets,105,Phoenix Suns,110,https://www.basketball-reference.com//boxscore...,NaN,"17,071",Footprint Center,NaN,0.0
15826,2024-02-29,10:00p,Miami Heat,97,Denver Nuggets,103,https://www.basketball-reference.com//boxscore...,NaN,"19,634",Ball Arena,NaN,0.0


In [13]:
rating = NBARater()
rating.eloSimulator(df)

In [14]:
teams_elo = rating.getTeams()

In [15]:
# Create traces for each key
traces = []
for key, values in teams_elo.items():
    trace = go.Scatter(x=list(range(1, len(values) + 1)), y=values, mode='lines+markers', name=key)
    traces.append(trace)

# Create layout
layout = go.Layout(title='Interactive Line Plot', xaxis=dict(title='Index'), yaxis=dict(title='Values'))

# Create figure
fig = go.Figure(data=traces, layout=layout)

# Show the interactive plot
fig.show()

In [ ]:
latest_rating = {}
for key, value in teams_elo.items():
    latest_rating[key] = value[-1]
dict(sorted(latest_rating.items(), key=lambda item: item[1], reverse=True))

In [ ]:
import requests
import bs4

url = f'https://en.wikipedia.org/wiki/2020_NBA_playoffs'
response = requests.get(url)
response.raise_for_status()

soup = bs4.BeautifulSoup(response.text, 'lxml')
dates = (soup.find('body')
         .find('div', class_='mw-page-container')
         .find('div', class_='mw-page-container-inner')
         .find('div', class_='mw-content-container')
         .find('main', class_='mw-body')
         .find('div', class_='mw-body-content')
         .find('div', class_='mw-content-ltr mw-parser-output')
         .find('table', class_='infobox vcard')
         .find('tbody')
         .find_all('tr')[3]
         .find('td', class_='infobox-data')).text
dates

In [ ]:
date = dates.split(',')
date

In [ ]:
date[0].split('–')[0].strip()